In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon, Polygon
import os
import sqlite3

import subprocess
from tqdm import tqdm

対象範囲を切り出すために，まず外接多角形の.polyを作成

In [23]:
# 都道府県名と地方名の対応辞書
region_mapping = {
    "北海道": "北海道地方",
    "青森県": "東北地方",
    "岩手県": "東北地方",
    "宮城県": "東北地方",
    "秋田県": "東北地方",
    "山形県": "東北地方",
    "福島県": "東北地方",
    "茨城県": "関東地方",
    "栃木県": "関東地方",
    "群馬県": "関東地方",
    "埼玉県": "関東地方",
    "千葉県": "関東地方",
    "東京都": "関東地方",
    "神奈川県": "関東地方",
    "新潟県": "中部地方",
    "富山県": "中部地方",
    "石川県": "中部地方",
    "福井県": "中部地方",
    "山梨県": "中部地方",
    "長野県": "中部地方",
    "岐阜県": "中部地方",
    "静岡県": "中部地方",
    "愛知県": "中部地方",
    "三重県": "近畿地方",
    "滋賀県": "近畿地方",
    "京都府": "近畿地方",
    "大阪府": "近畿地方",
    "兵庫県": "近畿地方",
    "奈良県": "近畿地方",
    "和歌山県": "近畿地方",
    "鳥取県": "中国地方",
    "島根県": "中国地方",
    "岡山県": "中国地方",
    "広島県": "中国地方",
    "山口県": "中国地方",
    "徳島県": "四国地方",
    "香川県": "四国地方",
    "愛媛県": "四国地方",
    "高知県": "四国地方",
    "福岡県": "九州地方",
    "佐賀県": "九州地方",
    "長崎県": "九州地方",
    "熊本県": "九州地方",
    "大分県": "九州地方",
    "宮崎県": "九州地方",
    "鹿児島県": "九州地方",
    "沖縄県": "九州地方"
}
def geodataframe_to_poly_files(gdf, output_directory):
    for index, row in gdf.iterrows():
        geometry = row['geometry']
        jcode = row["JCODE"]
        filename = f"{output_directory}/{jcode}.poly"

        with open(filename, 'w') as poly_file:
            if geometry.geom_type == 'Polygon':
                poly_file.write(f"polygon_{index}\n")
                exterior_coords = list(geometry.exterior.coords)
                for coord in exterior_coords:
                    poly_file.write(f"    {coord[0]} {coord[1]}\n")
                poly_file.write("END\n\n")
            elif geometry.geom_type == 'MultiPolygon':
                for i, polygon in enumerate(geometry.geoms):
                    poly_file.write(f"polygon_{index}_{i}\n")
                    exterior_coords = list(polygon.exterior.coords)
                    for coord in exterior_coords:
                        poly_file.write(f"    {coord[0]} {coord[1]}\n")
                    poly_file.write("END\n\n")
DIR = "c:/lab/accessibility/"

In [10]:
"""市区町村向け"""
path_to_edge = DIR+"japan_ver84/japan_ver84.shp"
path_to_output_poly = DIR+"gov_poly"
raw_gdf = gpd.read_file(path_to_edge)
gdf = raw_gdf.copy()

In [17]:
"""任意の範囲にクエリ"""
target_jcode = "14100"
target = gdf.query("SEIREI == '横浜市'").dissolve(by='SEIREI',as_index=False)
target["JCODE"] = pd.Series([target_jcode])
gdf = target.filter(items=["JCODE", "geometry"])
gdf_convex = gdf.copy()
gdf_convex['geometry'] = gdf_convex['geometry'].apply(lambda geom: geom.convex_hull) # 外接多角形に置換

In [24]:
out_path = "C:/Users/tora2/downloads/"
geodataframe_to_poly_files(gdf_convex,out_path)

.polyをもとに，pbfを切り抜く

In [26]:
pbf_file = f"/mnt/c/lab/accessibility/separate_road_pbf_drop_pf/14-road.osm.pbf"
poly_file = f"/mnt/c/lab/accessibility/gov_poly/additional/{target_jcode}.poly"
output_file = f"/mnt/c/lab/accessibility/separate_road_pbf_drop_pf/additional/{target_jcode}-road.osm.pbf"

# osmconvertコマンドの作成
osmconvert_path = "/usr/bin/osmconvert"  # osmconvertのパス
# command = f"{osmconvert_path} {pbf_file} -B={poly_file} --complete-ways -o={output_file}"
command = f"{osmconvert_path} {pbf_file} -B={poly_file} -o={output_file}"
result = subprocess.run(["wsl", "bash", "-c", command], capture_output=True, text=True)
if result.returncode != 0:
    print(f"Error @ {pbf_file} \n {poly_file}")